In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Function to scrape books from a category page
def scrape_category(category_url):
    book_names = []
    book_prices = []
    book_ratings = []
    book_stock = []  # Add a new list for stock status

    while category_url:
        # Send an HTTP GET request to the category page
        category_response = requests.get(category_url)

        # Check if the request was successful
        if category_response.status_code == 200:
            # Parse the HTML content of the category page
            category_soup = BeautifulSoup(category_response.text, "html.parser")

            # Find book names, prices, ratings, and stock status
            books = category_soup.find_all("h3")
            prices = category_soup.find_all("p", class_="price_color")
            ratings = category_soup.find_all("p", class_=lambda x: x and x.startswith("star-rating"))
            stock_status = category_soup.find_all("p", class_="availability")

            # Append data to the lists
            for book, price, rating, stock in zip(books, prices, ratings, stock_status):
                book_names.append(book.a.attrs["title"])
                book_prices.append(price.text.strip())
                # Extract the rating from the class attribute
                book_ratings.append(rating.attrs["class"][1])
                book_stock.append(stock.text.strip())  # Append stock status to the main list

            # Check if there is a "Next" button
            next_button = category_soup.find("li", class_="next")
            if next_button:
                next_page_href = next_button.a["href"]
                category_url = category_url.rsplit("/", 1)[0] + "/" + next_page_href
            else:
                category_url = None
        else:
            category_url = None

    return book_names, book_prices, book_ratings, book_stock


In [3]:
# URL of the website
url = "https://books.toscrape.com/index.html"

# Create empty lists to store data
book_categories = []
book_names = []
book_prices = []
book_ratings = []
book_stock = []

# Send an HTTP GET request to the website
response = requests.get(url)

In [4]:
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, "html.parser")

    # Find all categories with names ending in 'y'
    categories = soup.select("ul.nav-list > li > ul > li > a")
    filtered_categories = [category for category in categories if category.text.strip().endswith("y")]

    # Iterate through the filtered categories
    for category in filtered_categories:
        category_name = category.text.strip()
        # Get the URL of the category
        category_url = url.replace("index.html", category.get("href"))

        # Scrape books from the category page (including pagination)
        category_book_names, category_book_prices, category_book_ratings, category_book_stock = scrape_category(category_url)

        # Append data to the main lists
        book_categories.extend([category_name] * len(category_book_names))
        book_names.extend(category_book_names)
        book_prices.extend(category_book_prices)
        book_ratings.extend(category_book_ratings)
        book_stock.extend(category_book_stock)   # Append stock status to the main list

In [5]:
# Create a DataFrame from the lists
books = pd.DataFrame({"Category": book_categories, "Book Name": book_names, "Price": book_prices, "Rating": book_ratings, 'Stock Status': book_stock})

In [6]:
books.head()

,Category,Book Name,Price,Rating,Stock Status
0,Mystery,Sharp Objects,Â£47.82,Four,In stock
1,Mystery,"In a Dark, Dark Wood",Â£19.63,One,In stock
2,Mystery,The Past Never Ends,Â£56.50,Four,In stock
3,Mystery,A Murder in Time,Â£16.64,One,In stock
4,Mystery,The Murder of Roger Ackroyd (Hercule Poirot #4),Â£44.10,Four,In stock


In [7]:
books.describe()

,Category,Book Name,Price,Rating,Stock Status
count,158,158,158,158,158
unique,10,157,155,5,1
top,Fantasy,The Star-Touched Queen,Â£56.02,One,In stock
freq,48,2,2,38,158


In [8]:
rating_mapping = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
books['Rating'] = books['Rating'].map(rating_mapping)

In [9]:
# Create a DataFrame from the lists
books1 = pd.DataFrame({"Category": book_categories, "Book Name": book_names, "Price": book_prices, "Rating": book_ratings, 'Stock Status': book_stock})

In [10]:
books1.head()

,Category,Book Name,Price,Rating,Stock Status
0,Mystery,Sharp Objects,Â£47.82,Four,In stock
1,Mystery,"In a Dark, Dark Wood",Â£19.63,One,In stock
2,Mystery,The Past Never Ends,Â£56.50,Four,In stock
3,Mystery,A Murder in Time,Â£16.64,One,In stock
4,Mystery,The Murder of Roger Ackroyd (Hercule Poirot #4),Â£44.10,Four,In stock


In [11]:
books.columns

Index(['Category', 'Book Name', 'Price', 'Rating', 'Stock Status'], dtype='object')

In [12]:
books['Price'] = books['Price'].str.replace('Â£', '').astype(float)

In [13]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Category      158 non-null    object 
 1   Book Name     158 non-null    object 
 2   Price         158 non-null    float64
 3   Rating        158 non-null    int64  
 4   Stock Status  158 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 6.3+ KB


In [14]:
books.head()

,Category,Book Name,Price,Rating,Stock Status
0,Mystery,Sharp Objects,47.82,4,In stock
1,Mystery,"In a Dark, Dark Wood",19.63,1,In stock
2,Mystery,The Past Never Ends,56.50,4,In stock
3,Mystery,A Murder in Time,16.64,1,In stock
4,Mystery,The Murder of Roger Ackroyd (Hercule Poirot #4),44.10,4,In stock


In [15]:
import pandas as pd

# Assuming you have your data in a DataFrame called 'df'

# Function to calculate tax based on the price
def calculate_tax(price):
    if price > 50:
        return price * 0.06  # 6% tax rounded to 2 decimal places
    elif price > 30:
        return price * 0.03  # 3% tax rounded to 2 decimal places
    else:
        return 0.00  # No tax if price is <= 30

# Apply the calculate_tax function to create a new 'Tax' column
books['Tax'] = books['Price'].apply(calculate_tax)

# Modify the 'Price (incl. tax)' column by adding the calculated tax
books['Price (incl. tax)'] = books['Price'] + books['Tax']




In [16]:
books.head()

,Category,Book Name,Price,Rating,Stock Status,Tax,Price (incl. tax)
0,Mystery,Sharp Objects,47.82,4,In stock,1.4346,49.2546
1,Mystery,"In a Dark, Dark Wood",19.63,1,In stock,0.0000,19.6300
2,Mystery,The Past Never Ends,56.50,4,In stock,3.3900,59.8900
3,Mystery,A Murder in Time,16.64,1,In stock,0.0000,16.6400
4,Mystery,The Murder of Roger Ackroyd (Hercule Poirot #4),44.10,4,In stock,1.3230,45.4230


In [17]:
# Save the DataFrame to an Excel file
books.to_csv("books_Y_SideCategory.csv", index=False)

In [18]:
import mysql.connector

# Create a MySQL connection
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Syark1324#",
    database="books"
)

# Create a cursor
cursor = connection.cursor()

# Insert data into the MySQL table
for index, row in books.iterrows():
    category = row['Category']
    book_name = row['Book Name']
    price = row['Price']
    rating = row['Rating']
    tax = row['Tax']
    price_with_tax = row['Price (incl. tax)']
    stock_status = row['Stock Status']


    # Insert data into the table
    cursor.execute(
        "INSERT INTO books (category, book_name, price, rating, tax, price_with_tax, stock_status) VALUES (%s, %s, %s, %s, %s, %s, %s)",
        (category, book_name, price, rating, tax, price_with_tax, stock_status)
    )

# Commit the changes and close the connection
connection.commit()
connection.close()
